![](assets/solutions-microsoft-logo-small.png)
<img src="assets/ai.jpg" style="height:200px;float:right;vertical-align:text-top">

## Artificial Intelligence on IaaS++

This is part 5 of a 7-part workshop. The Jupyter Notbooks we are using are arranged in the same order as the Team Data Science Process: 

0 - [Introduction and Setup](./0%20-%20Introduction.ipynb)

1 - [Business Understanding](./1%20-%20Business%20Understanding.ipynb)

2 - [Data Acquisition and Understanding](./2%20-%20Data%20Acquisition%20and%20Understanding.ipynb)

3 - [Modeling](./3%20-%20Modeling.ipynb)

4 - *(This Module)* [Deployment](./4%20-%20Deployment.ipynb)

5 - [Customer Acceptance](./5%20-%20Customer%20Acceptance.ipynb)

6 - [Workshop Wrap-up](./6%20-%20Workshop%20Wrap-up.ipynb)

<p style="border-bottom: 3px solid lightgrey;"></p> 

<h1><img style="float: left; margin: 0px 15px 15px 0px;" src="./assets/check.png">Phase Four - Deployment</h1>

In the previous notebook, we trained an LSTM neural net to predict aircraft engine failure 30 cycles into the future.

In this notebook, we will create the artifacts and scripts to deploy the LSTM model into a webservice on Azure. The artifacts include the model files, and test scripts to validate your model can be used to predict future reliability of the engines based on the present operating characteristics.

Read the [Documentation Reference here](https://docs.microsoft.com/en-us/azure/machine-learning/team-data-science-process/lifecycle-deployment)

**Goal**
 - Deploy models with a data pipeline to a production or production-like environment for final user acceptance

**How to do it**
  - Deploy the model and pipeline to a production or production-like environment for application consumption


<p><img style="float: right; margin: 0px 15px 15px 0px;" src="./assets/aml-logo.png"><b>Using Azure Machine Learning for this Phase:</b></p>

<p><img style="float: left; margin: 0px 15px 15px 0px;" src="./assets/checkbox.png">[Deploy models in production](https://docs.microsoft.com/en-us/azure/machine-learning/team-data-science-process/deploy-models-in-production)</p>
<p><img style="float: left; margin: 0px 15px 15px 0px;" src="./assets/checkbox.png">[Configure your environment to operationalize](https://docs.microsoft.com/en-us/azure/machine-learning/preview/cli-for-azure-machine-learning#o16n)</p>
<p><img style="float: left; margin: 0px 15px 15px 0px;" src="./assets/checkbox.png">[Model management](https://docs.microsoft.com/en-us/azure/machine-learning/preview/deployment-setup-configuration)</p>

<p style="border-bottom: 1px solid lightgrey;"></p> 

<p style="border-bottom: 1px solid lightgrey;"></p> 

### Lab 4.0 - Model Building & Evaluation

<img src="assets/checkmark.jpg" style="float:right;vertical-align:text-top">

Using the training and test data sets we constructed in the previous Jupyter notebook, this notebook builds a LSTM network for scenerio described at [Predictive Maintenance Template](https://gallery.cortanaintelligence.com/Collection/Predictive-Maintenance-Template-3) to predict failure in aircraft engines. We will store the model for deployment in an Azure web service which we build in the next Jupyter notebook. We'll start with setting up the environment for this phase:

Instructions:
 1. Run the cells below one at a time.

#### Lab verification
<p><img style="float: left; margin: 0px 15px 15px 0px;" src="./assets/checkbox.png">Ensure that each cell runs, and produces the correct prediction.</p>

In [ ]:
import keras
# import the libraries
import os
import pandas as pd
import numpy as np
import json
import shutil
from keras.models import model_from_json

import h5py

# For creating the deployment schema file
from azureml.api.schema.dataTypes import DataTypes
from azureml.api.schema.sampleDefinition import SampleDefinition
from azureml.api.realtime.services import generate_schema

# For Azure blob storage access
from azure.storage.blob import BlockBlobService
from azure.storage.blob import PublicAccess

### Model storage 

We will store the model and operationalization artifacts in an Azure Blob Storage Container for easy retrieval to your deployment platform. When you created your Data Science Virtual Machine, it created a corresponding Azure Storage Account. We can use this central location for the common model assets.

Instructions for setting up your Azure Storage account are available within this link (https://docs.microsoft.com/en-us/azure/storage/blobs/storage-python-how-to-use-blob-storage). 

You will need to copy your account name and account key from the _Access Keys_ area in the portal into the following code block. These credentials will be reused in all of this workshop's Jupyter notebooks.

We will handle creating the containers and writing the data to these containers for each notebook. Further instructions for using Azure Blob storage with AML Workbench are available (https://github.com/Azure/ViennaDocs/blob/master/Documentation/UsingBlobForStorage.md).

You will need to enter the **ACCOUNT_NAME** as well as the **ACCOUNT_KEY** in order to access Azure Blob storage account you have created. This notebook will create and store all the resulting data files in a blob container under this account:

In [ ]:
# Enter your Azure blob storage details here 
ACCOUNT_NAME = "<your blob storage account name>"

# You can find the account key under the _Access Keys_ link in the 
# [Azure Portal](portal.azure.com) page for your Azure storage container.
ACCOUNT_KEY = "<your blob storage account key>"

#-------------------------------------------------------------------------------------------
# The data from the Data Ingestion and Preparation notebook is stored in the sensordata ingestion container.
MODEL_CONTAINER = "pmlstmmodel" 
# Connect to your blob service     
az_blob_service = BlockBlobService(account_name=ACCOUNT_NAME, account_key=ACCOUNT_KEY)

These variables tell the notebook where to store and load files for building and testing the model:

In [ ]:
# We will store each of these data sets in a local persistance folder
SHARE_ROOT = '/gpuclass/data/'

# These file names detail the data files. 
TEST_DATA = 'PM_test_files.pkl'

# We'll serialize the model in json format
LSTM_MODEL = 'modellstm.json'

# and store the weights in h5
MODEL_WEIGHTS = 'modellstm.h5'

# and the schema file
SCHEMA_FILE = 'service_schema.json'

### Load the test data frame

We have previously stored the test data frame in the local persistence directory indicated by the `SHARE_ROOT` variable. We'll use this test data frame to build the model schema to describe the deployment function calls:

In [ ]:
test_df = pd.read_pickle(SHARE_ROOT + TEST_DATA)
test_df.head(10)

We will need to recreate the feature engineering (creating the sequence features) just as we did in the model building step. We will do this within the webservice so that the service can take the raw sensor data, and return a scored result predicting probability of failure at 30 days (`label1`). 

When scoring an unseen observation, the model will not know the true labels. Therefore, we create a `score_df` without labels:

In [ ]:
# pick the feature columns 
# Sequence help order the observations in "time"
sequence_cols = ['setting1', 'setting2', 'setting3', 'cycle_norm']

# key columns group the machines
key_cols = ['id', 'cycle']

# Labels are what we're predicting.
label_cols = ['label1', 'label2', 'RUL']

# The scoreing data should not have labels... if we knew the label, 
# we wouldn'y need to predict.
score_df = test_df.drop(label_cols, axis = 1)

### Test init() and run() functions

The web service requires two functions, an `init()` function that will initialize the web service by loading the model into the service, and a `run()` function that will engineer the features to match the model call structure, and score that data set. We create the functions in here for testing and debugging:

In [ ]:
def init():
    # read in the model file
    from keras.models import model_from_json
    global loaded_model
    
    # load json and create model
    with open(SHARE_ROOT + LSTM_MODEL, 'r') as json_file:
        loaded_model_json = json_file.read()
        json_file.close()
        loaded_model = model_from_json(loaded_model_json)
    
    # load weights into new model
    loaded_model.load_weights(os.path.join(SHARE_ROOT, MODEL_WEIGHTS))
    loaded_model.compile('sgd','mse')

In [ ]:
def run(score_input): 
    # Create the sequences
    sequence_length = 50
    sequence_cols = ['setting1', 'setting2', 'setting3', 'cycle_norm']
    key_cols = ['id', 'cycle']

    # Feature engineering
    input_features = score_input.columns.values.tolist()
    sensor_cols = [x for x in input_features if x not in set(key_cols)]
    sensor_cols = [x for x in sensor_cols if x not in set(sequence_cols)]

    # The time is sequenced along
    # This may be a silly way to get these column names, but it's relatively clear
    sequence_cols.extend(sensor_cols)
    
    seq_array = [score_input[score_input['id']==id][sequence_cols].values[-sequence_length:] 
                 for id in score_input['id'].unique() if len(score_input[score_input['id']==id]) >= sequence_length]

    seq_array = np.asarray(seq_array).astype(np.float32)
    try:
        prediction = loaded_model.predict_proba(seq_array)
        #print(prediction)
        pred = prediction.tolist()
        return(pred)
    except Exception as e:
        return(str(e))

Check that there are 100 unique engine IDs: 

In [ ]:
print(score_df.id.unique())

The webservice test requires an `initialize` of the webservice, then send the entire scoring data set into the model. We expect to get 1 probability prediction for each engine in the scoring data set. Since the `score_df` has 100 machines, we expect 100 probabilities back:

In [ ]:
init()

prb=run(score_df)
print(prb)

*(Note: If you get an error in the last cell, wait a few moments and run it again)*

Instead we get 93, because 7 of the machines do not have the full 50 cycles available for scoring. If we send a machine with fewer than 50 records we get the following back: 

In [ ]:
tst_df=score_df.loc[score_df['id'] == 1]

print(tst_df.shape)

# Because 
run(tst_df)

If we send a complete data set, like machineID == 3, we get a probability back:

In [ ]:
tst_df=score_df.loc[score_df['id'] == 3]

print(tst_df.shape)

# Because 
ans=run(tst_df)

print(ans)

### Persist model assets

Next we persist the assets we have created for use in operationalization. First we need to define the schema so the webservice knows what the payload data will look like as it comes in:

In [ ]:
# define the input data frame
inputs = {"score_input": SampleDefinition(DataTypes.PANDAS, score_df)}

json_schema = generate_schema(run_func=run, inputs=inputs, filepath=SCHEMA_FILE)

# save the schema file for deployment
out = json.dumps(json_schema)
with open(SHARE_ROOT + SCHEMA_FILE, 'w') as f:
    f.write(out)

The conda dependencies are defined in this `webservices_conda.yaml` file. This will be used to tell the webservice server which python packages are required to run this web service:

In [ ]:
%%writefile {SHARE_ROOT}webservices_conda.yaml

# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for managed runs. These include runs against
# the localdocker, remotedocker, and cluster compute targets.

# Note that this file is NOT used to automatically manage dependencies for the
# local compute target. To provision these dependencies locally, run:
# conda env update --file conda_dependencies.yml

# Details about the Conda environment file format:
# https://conda.io/docs/using/envs.html#create-environment-file-by-hand

# For managing Spark packages and configuration, see spark_dependencies.yml.

name: project_environment
channels:
- conda-forge
- defaults
dependencies:
  - python=3.5.2
  - pip:
    - azure-common==1.1.8
    - azure-storage==0.36.0
    - numpy==1.14.0 
    - sklearn
    - keras
    - tensorflow
    - h5py

The `lstmscore.py` file is python code defining the web service operation. It includes both the `init()` and `run()` functions defined earlier imports the required libraries. These should be nearly identical to the previous defined versions:

In [ ]:
%%writefile {SHARE_ROOT}lstmscore.py

# import the libraries
import keras
import tensorflow
import json
import shutil
import numpy as np


def init():
    # read in the model file
    from keras.models import model_from_json
    global loaded_model
    
    # load json and create model
    with open('modellstm.json', 'r') as json_file:
        loaded_model_json = json_file.read()
        json_file.close()
        loaded_model = model_from_json(loaded_model_json)
    
    # load weights into new model
    loaded_model.load_weights("modellstm.h5")
    loaded_model.compile('sgd','mse')

def run(score_input):
    # Create the sequences
    sequence_length = 50
    sequence_cols = ['setting1', 'setting2', 'setting3', 'cycle_norm']
    key_cols = ['id', 'cycle']

    input_features = score_input.columns.values.tolist()
    sensor_cols = [x for x in input_features if x not in set(key_cols)]
    sensor_cols = [x for x in sensor_cols if x not in set(sequence_cols)]

    # The time is sequenced along
    # This may be a silly way to get these column names, but it's relatively clear
    sequence_cols.extend(sensor_cols)
    
    seq_array = [score_input[score_input['id']==id][sequence_cols].values[-sequence_length:] 
                 for id in score_input['id'].unique() if len(score_input[score_input['id']==id]) >= sequence_length]

    seq_array = np.asarray(seq_array).astype(np.float32)
    try:
        prediction = loaded_model.predict_proba(seq_array)
        
        pred = prediction.tolist()
        return(pred)
    except Exception as e:
        return(str(e))
    
if __name__ == "__main__":
    init()
    run("{\"score_df\": [{\"s20\": 0.5581395348837184, \"s10\": 0.0, \"s2\": 0.5451807228915584, \"s21\": 0.6618337475835432, \"s9\": 0.12761374854168395, \"s19\": 0.0, \"s3\": 0.31066056245912677, \"cycle\": 1, \"s15\": 0.3089649865332831, \"s4\": 0.2694125590817009, \"s1\": 0.0, \"s11\": 0.2083333333333357, \"cycle_norm\": 0.0, \"s13\": 0.2205882352941444, \"s5\": 0.0, \"s18\": 0.0, \"s8\": 0.2121212121210192, \"s14\": 0.1321601816492901, \"s6\": 1.0, \"setting2\": 0.75, \"setting1\": 0.632183908045977, \"s12\": 0.6460554371002161, \"s17\": 0.3333333333333357, \"s16\": 0.0, \"id\": 1, \"setting3\": 0.0, \"s7\": 0.6521739130434696}, {\"s20\": 0.6821705426356601, \"s10\": 0.0, \"s2\": 0.15060240963856586, \"s21\": 0.6868268434134208, \"s9\": 0.14668401687158195, \"s19\": 0.0, \"s3\": 0.37955090473076325, \"cycle\": 2, \"s15\": 0.21315890727203168, \"s4\": 0.2223160027008788, \"s1\": 0.0, \"s11\": 0.38690476190476275, \"cycle_norm\": 0.002770083102493075, \"s13\": 0.26470588235270043, \"s5\": 0.0, \"s18\": 0.0, \"s8\": 0.16666666666696983, \"s14\": 0.20476829394158358, \"s6\": 1.0, \"setting2\": 0.25, \"setting1\": 0.3448275862068965, \"s12\": 0.7398720682302695, \"s17\": 0.4166666666666714, \"s16\": 0.0, \"id\": 1, \"setting3\": 0.0, \"s7\": 0.8051529790660226}, {\"s20\": 0.7286821705426334, \"s10\": 0.0, \"s2\": 0.3765060240963862, \"s21\": 0.7213476940071786, \"s9\": 0.15808130664991182, \"s19\": 0.0, \"s3\": 0.34663178548071016, \"cycle\": 3, \"s15\": 0.4586379376683354, \"s4\": 0.3222484807562438, \"s1\": 0.0, \"s11\": 0.38690476190476275, \"cycle_norm\": 0.0055401662049861505, \"s13\": 0.2205882352941444, \"s5\": 0.0, \"s18\": 0.0, \"s8\": 0.22727272727297532, \"s14\": 0.15564041696769948, \"s6\": 1.0, \"setting2\": 0.5833333333333334, \"setting1\": 0.5172413793103449, \"s12\": 0.6993603411513902, \"s17\": 0.4166666666666714, \"s16\": 0.0, \"id\": 1, \"setting3\": 0.0, \"s7\": 0.6859903381642596}]}")

We also include a python file `test_service.py` which can test the web service you create. This program will test that there are enough cycles for the webservice to score, and send a single engine set to the web service. The results are printed with the engine ID to simulate how to use the webservice in a production setting.

After you create the web service, you will need to edit this file and provide the correct `url` endpoint string before running this program:

In [ ]:
%%writefile {SHARE_ROOT}test_service.py

import urllib
import json 
import requests
import pandas as pd

# The URL will need to be editted after service create.
url = 'http://127.0.0.1:32773/score'

## Sequence length will need to match the training sequence length from
## 2_model_building_and_evaluation.ipynb
sequence_length = 50

# We'll read in this data to test the service
test_df = pd.read_pickle('PM_test_files.pkl')

# Labels are what we're predicting.
label_cols = ['label1', 'label2', 'RUL']

# The scoreing data should not have labels... if we knew the label, 
# we wouldn't need to predict.
score_df = test_df.drop(label_cols, axis = 1)
headers = {'Content-Type':'application/json'}

# Now get the machine numbers, for each machine get the 
# prediction for the label timepoint
machineID = score_df['id'].unique()

for ind in machineID:
    
    try:
        body = score_df[score_df.id==ind]
        print('ID {}: size {}'.format(ind, body.shape))
        if body.shape[0] < sequence_length : 
            print("Skipping machineID {} as we need {} records to score and only have {} records.".format(ind, sequence_length, body.shape[0]))
            continue
        print('ID {}: {} \t {}'.format(ind, body.shape, body.tail(sequence_length+ 10).shape))
        body = "{\"score_input\": " + \
                    body.tail(sequence_length+10).to_json(orient="records") +\
                    "}"
        
        req = urllib.request.Request(url, str.encode(body), headers) 
        with urllib.request.urlopen(req) as response:
            the_page = response.read()
            print('ID {}: {}'.format(ind,the_page))
        
    except urllib.error.HTTPError as error:
        print("The request failed with status code {}: \n{}".format(error, error.read))

        # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
        print(error.info())
        print(error.reason)      

### Packaging

To move the model artifacts around, we'll zip them into one file. We can then retrieve this file from the persistence shared folder on your DSVM.

https://docs.microsoft.com/en-us/azure/machine-learning/preview/how-to-read-write-files

In [ ]:
# Compress the operationalization assets for easy blob storage transfer
# We can remove the persisted data files.
# !rm {SHARE_ROOT}/PM*.pkl
!rm {SHARE_ROOT}PM_train_files.pkl
!ls {SHARE_ROOT}

MODEL_O16N = shutil.make_archive('LSTM_o16n', 'zip', SHARE_ROOT)

# Create a new container if necessary, otherwise you can use an existing container.
# This command creates the container if it does not already exist. Else it does nothing.
az_blob_service.create_container(MODEL_CONTAINER,
                                 fail_on_exist=False, 
                                 public_access=PublicAccess.Container)

# Transfer the compressed operationalization assets into the blob container.
az_blob_service.create_blob_from_path(MODEL_CONTAINER, "LSTM_o16n.zip", str(MODEL_O16N)) 

<p style="border-bottom: 1px solid lightgrey;"></p> 

### Lab 4.1 - Operationalization

<img src="assets/checkmark.jpg" style="float:right;vertical-align:text-top">

Once the assets are stored, we can download them into a deployment compute context for operationalization on an Azure web service. For this scenario, we will deploy this on our local docker container context.

We demonstrate how to setup this web service this through an Azure ML CLI window. 

### Download the model

To download the model we've saved, follow these instructions on a local computer.

 - Open the Azure Portal
 - In the left hand pane, click on All resources
 - Search for the storage account using the name you provided earlier in this notebook.
 - Choose the storage account from search result list, this will open the storage account panel.
 - On the storage account panel, choose Blobs
 - On the Blobs panel choose the container `pmlstmmodel`
 - Select the file `LSTM_o16n.zip` and on the properties pane for that blob, choose download.

Once downloaded, unzip the file into the directory of your choosing. The zip file contains the deployment assets:

- the `lstmscore.py` file which contains functionst to do the model scoring
- the `modellstm.json` model definition file
- the `modellstm.h5` model weights file
- the `service_schema.json` which defines the input data schema
- the `webservices_conda.yaml` defining which python packages are required to run this service.
- the `test_service.py` script for testing your webservice once deployed.
- the `PM_test_df.pkl` test data set used by the `test_service.py` script

### Create a Model Management Account

Instructions:
1.  On your DSVM terminal connection, type the following commands:

`az login`

2. You will be given instructions for a web page and code to authorize the connection. Perform those actions. You will be given a login panel, use the Azure Account you started with for this Workshop.

3. Next, create an Azure Resource Group to hold all of the assets for this workshop with the following commands, entering the same region (which you can find in the Azure Portal) where you created your DSVM:
 
`az group create --name gpurg --location <ACCOUNT_REGION>`

*Note: You can find the correct text for the `<ACCOUNT_REGION>` using the command:*

`az account list-locations`

4. Now create a Model Management service in your Microsoft Azure account, called `gpumodelmanagement`. (You only need to do this once for any models you would like to deploy.) You will need to supply an **ACCOUNT_REGION** as before from your Azure account. The remaining defaults are acceptable. Run the following code:

`az ml account modelmanagement create --location <ACCOUNT_REGION> --resource-group gpurg --name gpumodelmanagement`

`az provider register -n Microsoft.MachineLearningCompute`

`az provider register -n Microsoft.ContainerRegistry`

`az provider register -n Microsoft.ContainerService`

*Note: If you get an error involving the region, read the error message carefully and select a region that is listed.*

*Note: If you get a `ResourceGroupNotFound` error, you may need to set the correct subscription. This is typically only an issue if your Azure login connects to multiple subscritpions.*

`az account set -s '<subscription name>'`

You can find the `subscription name` or `subscription id` through the (https://portal.azure.com) under the resource group you'd like to use. 

*Note: You can use these resources to learn more about AML and Docker:*

https://azure.github.io/LearnAI-Bootcamp/lab04.1-managing_models_with_aml/0_README

and

https://azure.github.io/LearnAI-Bootcamp/lab04.2-deploying_a_scoring_service_to_aks/0_README


#### Lab verification
<p><img style="float: left; margin: 0px 15px 15px 0px;" src="./assets/checkbox.png">Complete the steps that follow:</p>

1. Show the currently active environment:

`az ml env show`

2. You should get a message that the compute environment is not set. If so, activate the environment with your model management context (created above): 

`az ml env setup --location <ACCOUNT_REGION> --resource-group gpurg --name gpumodelmanagement`

...using the same `<ACCOUNT_REGION>` and `<RESOURCE_GROUP>` from the previous section. It will take a few minutes for this to complete. You can check the status with the command:

`az ml env show -g gpurg -n gpumodelmanagement` 

3. Then set the current environment:

`az ml env set --resource-group gpurg --cluster-name gpumodelmanagement`

*Note: if you get an error that it is not complete, wait a few minutes and re-run the command.*

3.  When that command completes successfully, check that the environment is now set:

`az ml env show` 
 
<p style="border-bottom: 3px solid lightgrey;"></p>


### Create a Docker Container for the Service

There are four steps to perform in order to create a container the service:

*NOTE: These commands assume the current directory contains the webservice assets we created in throughout the notebooks in this scenario (at least `lstmscore.py`, `modellstm.json`, `modellstm.h5`, `service_schema.json` and `webservices_conda.yaml`). If not, in the AML CLI window, change to the directory where the zip file was unpacked:*

`cd /gpuclass/data`

We start by registering the model files and add a description using -d arguments:

`az ml model register -m . -n gpuservice -d "GPU-trained model."`

We can see the model (along with other versions if we had previously registered models under the same name) by running

`az ml model list -o table`

Next we create a manifest for the model in the Azure Container Service. To do so, in the next command, we replace <MODEL_ID> with the model ID that was returned in the last command:

`az ml manifest create -n lstmwebservice -f lstmscore.py -s service_schema.json -r python -c webservices_conda.yaml -i <MODEL_ID>`

We now get the manifest ID when we run az ml manifest create. Make a note of this id and replace it in the below command when creating image:

`az ml service create realtime --image-id <image id> -n lstmwebservice`

Finally, the last step is to update the existing service out of the new image created. We would need the image ID created from the last step along with the service ID. To obtain the service id, we can run az ml service list realtime to get a list of all the service IDs, or we can look up the service on the Azure portal. Run the below command to update the service:

`az ml service create realtime --image-id <image id> -n lstmwebservice`

This command may a little time to complete.

## Test your deployment.

Once complete, the `az ml service create` command returns sample usage commands to test the service for both PowerShell and the cmd prompt. You can copy and paste this command into the CLI to test the web service. However, since it is only an example with 1 cycle, we know the model can not return a reasonable score. Instead you will see the following error as detailed above:
```
'Error when checking : expected lstm_1_input to have 3 dimensions, but got array with shape (0, 1)'

```
We have provided the `test_service.py` python script to send larger payloads to the service. First obtain the webservice endpoint with the following command. `az ml service usage realtime -i lstmwebservice`

```
> az ml service usage realtime -i lstmwebservice
Scoring URL:
    http://127.0.0.1:32770/score

Headers:
    Content-Type: application/json

Swagger URL:
    http://127.0.0.1:32770/swagger.json

Sample CLI command:
...
```

Copy the Scoring URL into the `url` variable in the `test_service.py` script and save the file.

From the CLI, you can then run the script with the command and response similar to the following:

`python test_service.py`

<p style="border-bottom: 1px solid lightgrey;"></p> 

### Lab 4.2 - (Optional) - Deploy to Edge IoT Device

<img src="assets/checkmark.jpg" style="float:right;vertical-align:text-top">

So far We have deployed the soltuion to a Docker container that you can host locally on the DSVM (as we have done here), remotely on another Data Science Virutal Machine, or to be used in a container service such as Azure AKS. 

There are stituations, such as in an airplane sensor, where the connection to the Prediction Service is not guarenteed. In this case you can delpoy the prediction engine to a small device, or to a full computer running the proper files. [Microsoft has a service called IoT Edge](https://docs.microsoft.com/en-us/azure/iot-fundamentals/) that is comprised of an IoT hub, an Edge runtime, and your prediction logic. 

Setting up an IoT edge involves the following steps:

  - [Install the configuration files on the Linux Edge device](https://docs.microsoft.com/en-us/azure/iot-edge/tutorial-simulate-device-linux#prerequisites)
  - [Create an IoT hub](https://docs.microsoft.com/en-us/azure/iot-edge/tutorial-simulate-device-linux#create-an-iot-hub)
  - [Register an IoT Edge device](https://docs.microsoft.com/en-us/azure/iot-edge/tutorial-simulate-device-linux#register-an-iot-edge-device)
  - [Install and start the IoT Edge runtime](https://docs.microsoft.com/en-us/azure/iot-edge/tutorial-simulate-device-linux#install-and-start-the-iot-edge-runtime)
  - [Deploy a module](https://docs.microsoft.com/en-us/azure/iot-edge/tutorial-simulate-device-linux#deploy-a-module)
  - [Create the Azure ML container](https://docs.microsoft.com/en-us/azure/iot-edge/tutorial-deploy-machine-learning#create-the-azure-ml-container)
  - [Add registry credentials to your Edge device](https://docs.microsoft.com/en-us/azure/iot-edge/tutorial-deploy-machine-learning#add-registry-credentials-to-your-edge-device)
  - [Run the solution](https://docs.microsoft.com/en-us/azure/iot-edge/tutorial-deploy-machine-learning#run-the-solution)
  - [View generated data](https://docs.microsoft.com/en-us/azure/iot-edge/tutorial-deploy-machine-learning#view-generated-data)

If you would like for all of these steps to be set up for you, see the following steps:

Instructions:
1. Open [this reference, and complete the steps you see there to deploy an IoT Edge solution](https://github.com/Azure/ai-toolkit-iot-edge/tree/master/Azure%20IoT%20Edge%20on%20DSVM). 


[More examples are available here](https://github.com/Azure/ai-toolkit-iot-edge )

*If you would like to read more information about this topic, check out these references:*
  - Set up Linux device as IoT Edge: https://docs.microsoft.com/en-us/azure/iot-edge/tutorial-simulate-device-linux 
  - ML Edge - https://docs.microsoft.com/en-us/azure/iot-edge/tutorial-deploy-machine-learning 

<p style="border-bottom: 3px solid lightgrey;"></p> 

### Phase 4 wrap-up

<img src="assets/wrapup.jpg" style="float:right;vertical-align:text-top">

<p>This module covered the Deployment phase of the solution.</p>

<p>The Notebooks are arranged in the same order as the Team Data Science Process:</p> 

0 - [Introduction and Setup](./0%20-%20Introduction.ipynb)

1 - [Business Understanding](./1%20-%20Business%20Understanding.ipynb)

2 - [Data Acquisition and Understanding](./2%20-%20Data%20Acquisition%20and%20Understanding.ipynb)

3 - [Modeling](./3%20-%20Modeling.ipynb)

4 - *(This module)* [Deployment](./4%20-%20Deployment.ipynb)

5 - *(Proceed to this Notebook Next)* [Customer Acceptance](./5%20-%20Customer%20Acceptance.ipynb)

6 - [Workshop Wrap-up](./6%20-%20Workshop%20Wrap-up.ipynb)